### Configuration

In [1]:
import os
import re
import numpy as np
import pandas as pd

import mne
from tqdm import tqdm
import datetime
import hdf5storage

import utils__config

In [2]:
os.chdir(utils__config.working_directory)
os.getcwd()

'Z:\\Layton\\Sleep_083023'

### Parameters

In [3]:
# dir_path = 'Z:/Layton/Sleep_083023/Cache/Subject01/S01_Feb02_wclus__1___ns5_to_mat'
# save_path = 'Cache/Subject01/S01_Feb02_micro_1024hz.fif'

# dir_path = 'Z:/Layton/Sleep_083023/Cache/Subject05/Jul11/S05_Jul11_wclus__1___ns5_to_mat'
# save_path = 'Cache/Subject05/Jul11/S05_Jul11_micro_1024hz.fif'

# dir_path = 'Z:/Layton/Sleep_083023/Cache/Subject05/Jul12/S05_Jul12_wclus__1___ns5_to_mat'
# save_path = 'Cache/Subject05/Jul12/S05_Jul12_micro_1024hz.fif'

dir_path = 'Z:/Layton/Sleep_083023/Cache/Subject05/Jul13/S05_Jul13_wclus__1___ns5_to_mat'
save_path = 'Cache/Subject05/Jul13/S05_Jul13_micro_1024hz.fif'

In [ ]:
stop_channel = 64
sampling_freq = 1024
n_jobs = -4

### Convert MAT to MNE Object

In [4]:
for channel in tqdm(os.listdir(dir_path)[:stop_channel]):

    # Load Data
    file_path = os.path.join(dir_path, channel)
    data = hdf5storage.loadmat(file_path)

    # Convert V to uV
    time_series = data['time_series'] * 1e-6

    # Channel Name
    ch_name = [re.search(r'Channel\d+', channel).group(0)]

    # Sampling Frequency
    sfreq = data['meta_data']['sampling_rate'].astype(np.int64)[0][0][0]

    # Start Time
    raw_time = data['meta_data']['time_stamp'][0][0][0][0].astype(np.int64)
    time_start = datetime.datetime(raw_time[0], raw_time[1], raw_time[3], 
                                   raw_time[4], raw_time[5], raw_time[6], 
                                   raw_time[7], tzinfo = datetime.timezone.utc)

    # Create Raw object
    info = mne.create_info(ch_names = ch_name,
                           sfreq = sfreq,
                           ch_types = 'seeg')

    single_raw = mne.io.RawArray(data = time_series,
                                 info = info)

    single_raw.set_meas_date(time_start)

    # Divide the recording in two if it's longer than 9.9 hours,
    # because such a recording at 30 kHz is too much data for the
    # MNE resample() function to handle.
    total_time = single_raw.times[-1]
    if total_time > 9.9 * 3600:  # 9.9 hours in seconds

        overlap = total_time * 0.1  # 10% overlap
        segment_length = total_time * 0.6  # 60% of total time

        # First Segment: 0 - 60%
        tmin = 0
        segment1 = single_raw.copy().crop(tmin=tmin, tmax=tmin + segment_length)
        segment1.resample(sfreq=sampling_freq)
        segment1 = segment1.crop(tmin=tmin, tmax=total_time * 0.5)  # Crop to 50%

        # Second Segment: 40% - 100%
        tmin = total_time * 0.4  # Start at 40% of total time
        segment2 = single_raw.copy().crop(tmin=tmin, tmax=total_time)
        segment2.resample(sfreq=sampling_freq)
        segment2 = segment2.crop(tmin=total_time * 0.5)  # Crop to 50%

        # Concatenate the two segments
        single_raw = mne.concatenate_raws([segment1, segment2])

        # Resample
        single_raw.resample(sfreq=sampling_freq)
        
    else:

        single_raw.resample(sfreq=sampling_freq)

    # Concatenate channels
    if ('raw' in locals() or 'raw' in globals()):
        raw.add_channels([single_raw])
    else:
        raw = single_raw.copy()

  0%|          | 0/64 [00:00<?, ?it/s]

Creating RawArray with float64 data, n_channels=1, n_times=1123200001
    Range : 0 ... 1123200000 =      0.000 ... 37440.000 secs
Ready.
Sampling frequency of the instance is already 1024.0, returning unmodified.


  2%|▏         | 1/64 [02:05<2:11:54, 125.63s/it]

Creating RawArray with float64 data, n_channels=1, n_times=1123200001
    Range : 0 ... 1123200000 =      0.000 ... 37440.000 secs
Ready.
Sampling frequency of the instance is already 1024.0, returning unmodified.


  3%|▎         | 2/64 [04:05<2:06:26, 122.37s/it]

...

 97%|█████████▋| 62/64 [2:07:16<04:11, 125.71s/it]

Creating RawArray with float64 data, n_channels=1, n_times=1123200001
    Range : 0 ... 1123200000 =      0.000 ... 37440.000 secs
Ready.
Sampling frequency of the instance is already 1024.0, returning unmodified.


 98%|█████████▊| 63/64 [2:09:22<02:05, 125.86s/it]

Creating RawArray with float64 data, n_channels=1, n_times=1123200001
    Range : 0 ... 1123200000 =      0.000 ... 37440.000 secs
Ready.
Sampling frequency of the instance is already 1024.0, returning unmodified.


100%|██████████| 64/64 [2:11:26<00:00, 123.23s/it]


Filter

In [7]:
# Bandpass filter 
raw.filter(l_freq = 0.3, h_freq = 256, n_jobs = n_jobs)

# Notch filter to remove 60 Hz line noise
raw.notch_filter(np.arange(60, sampling_freq/2, 60))

Filtering raw data in 2 contiguous segments
Setting up band-pass filter from 0.3 - 2.6e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.30
- Lower transition bandwidth: 0.30 Hz (-6 dB cutoff frequency: 0.15 Hz)
- Upper passband edge: 256.00 Hz
- Upper transition bandwidth: 64.00 Hz (-6 dB cutoff frequency: 288.00 Hz)
- Filter length: 11265 samples (11.001 s)



[Parallel(n_jobs=-4)]: Using backend LokyBackend with 29 concurrent workers.
[Parallel(n_jobs=-4)]: Done  49 out of  64 | elapsed:   22.5s remaining:    6.8s
[Parallel(n_jobs=-4)]: Done  62 out of  64 | elapsed:   24.3s remaining:    0.7s
[Parallel(n_jobs=-4)]: Done  64 out of  64 | elapsed:   24.5s finished


Filtering raw data in 2 contiguous segments
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 6759 samples (6.601 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    5.6s


Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 6759 samples (6.601 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    1.1s


<RawArray | 64 x 23003137 (22464.0 s), ~10.97 GB, data loaded>

Export

In [8]:
raw.save(save_path, overwrite = True)

Writing Z:\Layton\Sleep_083023\Cache\Subject05\Jul13\S05_Jul13_micro_1024hz.fif


C:\Users\lal85\AppData\Local\Temp\2\ipykernel_11508\2458318313.py:1: RuntimeWarning: This filename (Z:\Layton\Sleep_083023\Cache\Subject05\Jul13\S05_Jul13_micro_1024hz.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw.save(save_path, overwrite = True)


Overwriting existing file.
Writing Z:\Layton\Sleep_083023\Cache\Subject05\Jul13\S05_Jul13_micro_1024hz-1.fif
Overwriting existing file.
Writing Z:\Layton\Sleep_083023\Cache\Subject05\Jul13\S05_Jul13_micro_1024hz-2.fif
Closing Z:\Layton\Sleep_083023\Cache\Subject05\Jul13\S05_Jul13_micro_1024hz-2.fif
Closing Z:\Layton\Sleep_083023\Cache\Subject05\Jul13\S05_Jul13_micro_1024hz-1.fif
Closing Z:\Layton\Sleep_083023\Cache\Subject05\Jul13\S05_Jul13_micro_1024hz.fif
[done]
